In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir('/Users/chrissoria/Documents/Research/BICS_Political_Polarization/')

For this preliminary analysis, I will only use wave 2,4,6. \
Each wave's data was collected on 2020-06-17 - 2020-06-23, 2020-11-29 - 2020-12-16, 2021-05-12 - 2021-05-25.

In [3]:
df_w1 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave1_unweighted.csv')
df_w2 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave2_unweighted.csv')
df_w3 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave3_unweighted.csv')
df_w4 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave4_unweighted.csv')
df_w5 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave5_unweighted.csv')
df_w6 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave6_unweighted.csv')

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_64930/808921033.py:6: DtypeWarning: Columns (84) have mixed types. Specify dtype option on import or set low_memory=False.
  df_w6 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave6_unweighted.csv')


In [4]:
if "resp_employ" in df_w2.columns:
    df_w2["resp_employ"] = df_w2["resp_employ"].astype(str)

if "resp_employ" in df_w3.columns:
    df_w3["resp_employ"] = df_w3["resp_employ"].astype(str)    
    
if "resp_employ" in df_w4.columns:
    df_w4["resp_employ"] = df_w4["resp_employ"].astype(str)
    
if "resp_employ" in df_w5.columns:
    df_w5["resp_employ"] = df_w5["resp_employ"].astype(str)

if "resp_employ" in df_w6.columns:
    df_w6["resp_employ"] = df_w6["resp_employ"].astype(str)
    
df_w2['resp_occupation'] = df_w2['resp_occupation'].astype(str)
df_w4['resp_occupation'] = df_w4['resp_occupation'].astype(str)

df_combined = pd.concat([df_w2, df_w3, df_w4, df_w5, df_w6], ignore_index=True)

Below I'm converting the waves into date ranges

In [5]:
print(df_w2.columns)

Index(['StartDate', 'EndDate', 'Status', 'Progress', 'Duration (in seconds)',
       'Finished', 'RecordedDate', 'ResponseId', 'DistributionChannel',
       'UserLanguage',
       ...
       'orig_education', 'w_educ', 'w_urbanrural', 'w_urbanrural2',
       'estimated_county_fips', 'estimated_county', 'estimated_state',
       'resp_hhsize_text', 'resp_hhsize_raw', 'resp_hhsize_topcode_val'],
      dtype='object', length=165)


In [6]:
print(df_w6['interview_date'].value_counts())

interview_date
2021-05-22    438
2021-05-23    432
2021-05-15    426
2021-05-16    426
2021-05-18    420
2021-05-19    418
2021-05-25    417
2021-05-17    415
2021-05-21    415
2021-05-24    407
2021-05-14    399
2021-05-13    396
2021-05-20    392
2021-05-26     17
Name: count, dtype: int64


In [7]:
df_combined['data_collected_dates'] = df_combined.apply(lambda row: "June 18-24, '20" if row['wave'] == 2
                                                       else "September 12-20, '20'" if row['wave'] == 3
                                                       else "December 1-7, '20'" if row['wave'] == 4
                                                       else "February 9-14, '21'" if row['wave'] == 5
                                                       else "May 14-25, '21'" if row['wave'] == 6
                                                        else np.nan, axis = 1)

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_64930/1458976774.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined['data_collected_dates'] = df_combined.apply(lambda row: "June 18-24, '20" if row['wave'] == 2


In [8]:
print(df_combined['data_collected_dates'].value_counts())
print(df_combined['wave'].value_counts())

data_collected_dates
May 14-25, '21'          5418
September 12-20, '20'    3248
December 1-7, '20'       2993
February 9-14, '21'      2976
June 18-24, '20          2432
Name: count, dtype: int64
wave
6    5418
3    3248
4    2993
5    2976
2    2432
Name: count, dtype: int64


We're missing a "conservative" but we have two moderates?

In [9]:
political_view_mapping = {
    'Extremely conservative': 1,
    'Slightly conservative': 2,
    'Moderate': 3,
    'Middle of the road': 3,
    'Slightly liberal': 4,
    'Liberal': 5,
    'Extremely liberal': 6
}

# Creating a new variable 'political_view_numeric' by mapping the 'political_view' column using the defined mapping
df_combined['political_view_numeric'] = df_combined['political_view'].replace(political_view_mapping)

df_combined['political_view'].value_counts()

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_64930/2967877890.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined['political_view_numeric'] = df_combined['political_view'].replace(political_view_mapping)


political_view
Moderate                  4542
Liberal                   2695
Slightly conservative     2277
Extremely liberal         2262
Extremely conservative    2021
Middle of the road        1779
Slightly liberal          1491
Name: count, dtype: int64

In [10]:
#education variable
def categorize_education(row):
    if row['resp_educ'] == "Less than high school degree":
        return "Less than high school"
    elif row['resp_educ'] in ["High school graduate (high school diploma or equivalent including GED)", "Some college but no degree", "Associate degree in college (2-year)"]:
        return "High school graduate"
    elif row['resp_educ'] in ["Bachelor's degree in college (4-year)", "Master's degree", "Doctoral degree", "Professional degree (JD, MD)"]:
        return "College graduate and above"
    else:
        return "Unknown"

df_combined['educ_group'] = df_combined.apply(categorize_education, axis=1)

df_combined['educ_group'].value_counts(dropna=False)

race_columns = [col for col in df_combined.columns if col.startswith('resp_race_')]
df_combined[race_columns] = df_combined[race_columns].fillna(0)

#race variable
def categorize_race(row):
    if row['resp_race_1'] == "White":
        return "White"
    elif row['resp_race_2'] == "Black or African American":
        return "Black"
    elif row['resp_race_4'] == "Asian":
        return "Asian"
    else:
        return "Other / Mixed"

df_combined['r_race'] = df_combined.apply(categorize_race, axis=1)

print(df_combined['r_race'].value_counts(dropna=False))

#employment variable
employment_columns = [col for col in df_combined.columns if col.startswith('resp_employ_')]
df_combined[employment_columns] = df_combined[employment_columns].fillna(0)

# Create the 'r_working' var based whether reported
df_combined['r_working'] = np.where(df_combined['industry'] == "I don't work", "Not Working", "Working")

print(df_combined['r_working'].value_counts(dropna=False))

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_64930/3210909615.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined['educ_group'] = df_combined.apply(categorize_education, axis=1)
/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_64930/3210909615.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined['r_race'] = df_combined.apply(categorize_race, axis=1)


r_race
White            13226
Black             1882
Asian             1180
Other / Mixed      779
Name: count, dtype: int64
r_working
Working        11372
Not Working     5695
Name: count, dtype: int64


/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_64930/3210909615.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined['r_working'] = np.where(df_combined['industry'] == "I don't work", "Not Working", "Working")


dependent variable for concern

In [11]:
df_combined['binary_concern'] = df_combined['covid19_concern'].apply(
    lambda x: 1 if x in ["Somewhat concerned", "Very concerned"] else (0 if pd.notna(x) else None)
)

df_combined['binary_concern_strong'] = df_combined['covid19_concern'].apply(
    lambda x: 1 if x == "Very concerned" else (0 if pd.notna(x) else None)
)

df_combined['contact_reduction'] = df_combined['covid19_f2fchange'].apply(
    lambda x: 1 if x == "I have greatly reduced face-to-face interaction with others" else (0 if pd.notna(x) else None)
)

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_64930/3902376750.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined['binary_concern'] = df_combined['covid19_concern'].apply(
/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_64930/3902376750.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined['binary_concern_strong'] = df_combined['covid19_concern'].apply(
/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_64930/3902376750.py:9: PerformanceWarning: DataFrame is highly fragmente

reading in the alter file

In [12]:
df_w2_nonhhalters = pd.read_csv("/Users/chrissoria/documents/research/BICs/data/national_alters_nonhh_wave2_unweighted.csv")
df_w3_nonhhalters = pd.read_csv("/Users/chrissoria/documents/research/BICs/data/national_alters_nonhh_wave3_unweighted.csv")
df_w4_nonhhalters = pd.read_csv("/Users/chrissoria/documents/research/BICs/data/national_alters_nonhh_wave4_unweighted.csv")
df_w5_nonhhalters = pd.read_csv("/Users/chrissoria/documents/research/BICs/data/national_alters_nonhh_wave5_unweighted.csv")
df_w6_nonhhalters = pd.read_csv("/Users/chrissoria/documents/research/BICs/data/national_alters_nonhh_wave6_unweighted.csv")

df_alters_combined = pd.concat([df_w2_nonhhalters, df_w3_nonhhalters, df_w4_nonhhalters, df_w5_nonhhalters, df_w6_nonhhalters], ignore_index=True)

what is num_cc_nonhh? \
Should I use this or how many reported contacts?

In [13]:
# Create binary indicators for mask, gloves, and other protective equipment usage
df_alters_combined['Mask_Used'] = (
    ((df_alters_combined['protection_1'] == "Wear a face mask") |
    (df_alters_combined['protection_2'] == "Wear a face mask") |
    (df_alters_combined['protection_3'] == "Wear a face mask") |
    (df_alters_combined['protection_4'] == "Wear a face mask")).astype(int)
)

df_alters_combined['Gloves_Used'] = (
    ((df_alters_combined['protection_1'] == "Wear gloves") |
    (df_alters_combined['protection_2'] == "Wear gloves") |
    (df_alters_combined['protection_3'] == "Wear gloves") |
    (df_alters_combined['protection_4'] == "Wear gloves")).astype(int)
)

df_alters_combined['Other_Protective_Equipment_Used'] = (
    ((df_alters_combined['protection_1'] == "Wear other protective equipment") |
    (df_alters_combined['protection_2'] == "Wear other protective equipment") |
    (df_alters_combined['protection_3'] == "Wear other protective equipment") |
    (df_alters_combined['protection_4'] == "Wear other protective equipment")).astype(int)
)

# Aggregate these indicators at the 'rid' level
df_aggregated = df_alters_combined.groupby('rid').agg(
    Total_Masks_Used=('Mask_Used', 'sum'),
    Total_Gloves_Used=('Gloves_Used', 'sum'),
    Total_Other_Protective_Equipment_Used=('Other_Protective_Equipment_Used', 'sum'),
    Contacts=('rid', 'size')
).reset_index()

# Normalize the counts by the number of contacts
df_aggregated['Norm_Masks_Used'] = df_aggregated['Total_Masks_Used'] / df_aggregated['Contacts']
df_aggregated['Norm_Gloves_Used'] = df_aggregated['Total_Gloves_Used'] / df_aggregated['Contacts']
df_aggregated['Norm_Other_Protective_Equipment_Used'] = df_aggregated['Total_Other_Protective_Equipment_Used'] / df_aggregated['Contacts']

# Calculate non-weighted and weighted safety indices
df_aggregated['Non_Weighted_Safety_Index'] = (
    df_aggregated['Norm_Masks_Used'] + 
    df_aggregated['Norm_Gloves_Used'] + 
    df_aggregated['Norm_Other_Protective_Equipment_Used']
) / 3

df_aggregated['Weighted_Safety_Index'] = (
    df_aggregated['Norm_Masks_Used'] + 
    df_aggregated['Norm_Gloves_Used'] + 
    df_aggregated['Norm_Other_Protective_Equipment_Used']
) / (3 * df_aggregated['Contacts'])

In [14]:
df_combined_contacts = pd.merge(df_combined, df_aggregated, on = "rid", how = "left")

In [15]:
df_combined_contacts['wave'].value_counts()

wave
6    5418
3    3248
4    2993
5    2976
2    2432
Name: count, dtype: int64

In [16]:
df_combined_contacts['political_party'].value_counts()

political_party
Democrat                7361
Republican              4814
Independent             3968
Prefer not to answer     924
Name: count, dtype: int64

In [17]:
df_combined_contacts.to_csv('data/BICS_ego_alters_merged.csv', index = False)

In [18]:
col_types = {
    'ZIP': str,
    'CD':str,
    'COUNTY_FIPS':str,
    'STATE_FIPS':str,
    'CONGRESSPERSON_PARTY': str
}

ZIP_FEATURES = pd.read_csv('/Users/chrissoria/Documents/Research/BICS_Political_Polarization/data/ZIP_Features.csv',
                          dtype=col_types)

In [20]:
col_types = {
    'resp_zip': str,
    'political_party': str
}

all_waves = df_combined_contacts

all_waves = all_waves.rename(columns={
    'resp_zip': 'ZIP'
})

columns_to_keep = ['ResponseId','StartDate','resp_yob','resp_sex','resp_hispanic','r_race','resp_nativity','ZIP',
                  'resp_hhsize','r_working','resp_occupation','lefthome_num','num_cc_nonhh',
                  'lefthome_where_1','lefthome_where_2','lefthome_where_3','lefthome_where_4','lefthome_where_10',
                  'lefthome_where_8','lefthome_where_9','lefthome_where_5','lefthome_where_11','lefthome_where_6',
                  'lefthome_where_7','inet_freq','socmedia_use','covid19_familiar','covid19_concern',
                  'covid19_f2fchange','covid19_reduceOK','policy_sip','age','hhi','political_party','political_view',
                  'industry','health_insurance','interview_date','wave','agecat','city','covid19_vax','covid19_whynot_vax',
                  'Non_Weighted_Safety_Index','Weighted_Safety_Index','Norm_Masks_Used','educ_group','contact_reduction',
                  'binary_concern','binary_concern_strong','resp_educ','resp_sex','Contacts','Non_Weighted_Safety_Index',
                  'Weighted_Safety_Index','data_collected_dates']
all_waves = all_waves[columns_to_keep]

all_waves['political_party'].value_counts()

KeyError: "['COUNTY_FIPS'] not in index"

In [ ]:
BICS_ZIP_FEATURES = pd.merge(all_waves, ZIP_FEATURES, on = "ZIP", how = "left")
BICS_ZIP_FEATURES = BICS_ZIP_FEATURES.drop_duplicates(subset='ResponseId', keep='first')
BICS_ZIP_FEATURES

In [ ]:
BICS_ZIP_FEATURES['political_party'].value_counts()

In [ ]:
BICS_ZIP_FEATURES['political_party_to_CD'] = BICS_ZIP_FEATURES.apply(
    lambda row: str(row['political_party']) + " in " + str(row['CONGRESSPERSON_PARTY']) + " led CD", axis=1
)

# Print the updated DataFrame
print(BICS_ZIP_FEATURES[['political_party', 'CONGRESSPERSON_PARTY', 'political_party_to_CD']])

In [ ]:
BICS_ZIP_FEATURES['political_party'].value_counts()

In [ ]:
BICS_ZIP_FEATURES['CONGRESSPERSON_PARTY'] = BICS_ZIP_FEATURES['CONGRESSPERSON_PARTY'].replace("Democratic", "Democrat")
BICS_ZIP_FEATURES['In_Opposing_Party_CD'] = (BICS_ZIP_FEATURES['political_party'] != BICS_ZIP_FEATURES['CONGRESSPERSON_PARTY']).astype(int)
BICS_ZIP_FEATURES['Independent'] = (BICS_ZIP_FEATURES['political_party'] == "Independent").astype(int)
BICS_ZIP_FEATURES['Vaccinated'] = BICS_ZIP_FEATURES['covid19_vax'].apply(
    lambda x: 1 if x == "Yes, I have received at least one dose of a vaccine" else
              (0 if x == "No, I have not" else np.nan)
)

Does it make sense to drop Independents? \
They're kind of in their own cateogory

In [ ]:
print(BICS_ZIP_FEATURES['In_Opposing_Party_CD'].value_counts())
print(BICS_ZIP_FEATURES['political_party'].value_counts())
print(BICS_ZIP_FEATURES['CONGRESSPERSON_PARTY'].value_counts())

Below I convert the raw numbers to categories for Trump and Biden share

In [ ]:
BICS_ZIP_FEATURES['Categorical_Trump_County_Share'] = BICS_ZIP_FEATURES['trump_percentage_won'].apply(lambda x:
    "Less Than a Third" if x < 33.1 else
    "Less than Two Thirds Greater Than One Third" if x < 66.1 else
    "Greater than or Equal to Two thirds" if x >= 66.1 else pd.NA
)

BICS_ZIP_FEATURES['Categorical_Biden_County_Share'] = BICS_ZIP_FEATURES['biden_percentage_won'].apply(lambda x:
    "Less Than a Third" if x < 33.1 else
    "Less than Two Thirds Greater Than One Third" if x < 66.1 else
    "Greater than or Equal to Two thirds" if x >= 66.1 else pd.NA
)

BICS_ZIP_FEATURES['Percentage_Trump_Greater_Than_Two_Thirds'] = BICS_ZIP_FEATURES['trump_percentage_won'].apply(lambda x:
    1 if x >= 66.1 else
    0 if x < 66.1 else pd.NA
)

BICS_ZIP_FEATURES['Percentage_Trump_Less_Than_Two_Thirds'] = BICS_ZIP_FEATURES['trump_percentage_won'].apply(lambda x:
    1 if x < 66.1 else
    0 if x <= 66.1 else pd.NA
)

BICS_ZIP_FEATURES['Percentage_Biden_Greater_Than_Two_Thirds'] = BICS_ZIP_FEATURES['biden_percentage_won'].apply(lambda x:
    1 if x >= 66.1 else
    0 if x < 66.1 else pd.NA
)

BICS_ZIP_FEATURES['Percentage_Biden_Less_Than_Two_Thirds'] = BICS_ZIP_FEATURES['biden_percentage_won'].apply(lambda x:
    1 if x < 66.1 else
    0 if x <= 66.1 else pd.NA
)

Next, I use the same logic to create the CD categoricals

In [ ]:
BICS_ZIP_FEATURES['Categorical_Repub_CD_County_Share'] = BICS_ZIP_FEATURES['CD_PERCENT_REPUBLICAN'].apply(lambda x:
    "Less Than a Quarter" if x < .25 else
    "Two Quarters" if x < .50 and x >.25 else
    "Three Quarters" if x < .75 and x > .50 else
    "Greater than or Equal to Three Quarters" if x >= .75 else pd.NA
)

BICS_ZIP_FEATURES['Categorical_Dem_CD_County_Share'] = BICS_ZIP_FEATURES['CD_PERCENT_DEMOCRAT'].apply(lambda x:
    "Less Than a Quarter" if x < .25 else
    "Two Quarters" if x < .50 else
    "Three Quarters" if x < .75 else
    "Greater than or Equal to Three Quarters" if x >= .75 else pd.NA
)

BICS_ZIP_FEATURES['Categorical_Repub_CD_County_Share'].value_counts()

Next, let's add a variable that treats urban and rural as a binary. \
1 = Metro - Counties in metro areas of 1 million population or more \
2 = Metro - Counties in metro areas of 250,000 to 1 million population \         
3 = Metro - Counties in metro areas of fewer than 250,000 population \
4 = Nonmetro - Urban population of 20,000 or more, adjacent to a metro area \
5 = Nonmetro - Urban population of 20,000 or more, not adjacent to a metro area \
6 = Nonmetro - Urban population of 2,500 to 19,999, adjacent to a metro area \
7 = Nonmetro - Urban population of 2,500 to 19,999, not adjacent to a metro area \
8 = Nonmetro - Completely rural or less than 2,500 urban population, adjacent to a metro area \
9 = Nonmetro - Completely rural or less than 2,500 urban population, not adjacent to a metro area

In [ ]:
BICS_ZIP_FEATURES['COUNT_RUCC_CAT'] = BICS_ZIP_FEATURES['COUNTY_RUCC_2013'].apply(
    lambda x: "Urban Metro" if x in [1, 2] else
              "Urban Nonmetro" if x in list(range(3, 8)) else 
              "Rural" if x in [8, 9] else np.nan
)

BICS_ZIP_FEATURES['COUNT_RUCC_BINARY'] = BICS_ZIP_FEATURES['COUNTY_RUCC_2013'].apply(
    lambda x: "Urban" if x in list(range(1, 8)) else  
              "Rural" if x in [8, 9] else np.nan
)

In [ ]:
BICS_ZIP_FEATURES.to_csv('data/BICS_ZIP_Features.csv',index=False)

In [ ]:
BICS_ZIP_FEATURES.to_csv('data/Partisanship and Health Behavior/Data/BICS_ZIP_Features.csv',index=False)